# Answering Business Questions Using SQL

We'll be working with a modified version of a database called *Chinook*. The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers' purchases. This information is contained in eleven tables. Here's a *schema diagram* for the Chinook database:

![Image](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

Let's start by getting familiar with our data.

In [2]:
%%sql

SELECT name,
       type  
  FROM sqlite_master
 WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label specialised in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA.

All four albums below are by artists that don't have any tracks in the store right now.

|      Artist Name     |  Genre  |
|:--------------------:|:-------:|
|         Regal        | Hip-Hop |
|       Red Tone       |   Punk  |
| Meteor and the Girls |   Pop   |
|    Slim Jim Bites    |  Blues  |


We will select three albums to be added to the Chinook store based on the genres that sell the best in the USA.

In [3]:
%%sql

WITH usa_tracks AS --Getting the genres and the tracks quantities filtered by the USA
         (
          SELECT g.name AS genre,
                 il.quantity
            FROM track AS t
           INNER JOIN genre AS g
              ON t.genre_id = g.genre_id
           INNER JOIN invoice_line AS il
              ON il.track_id = t.track_id
           INNER JOIN invoice AS i
              ON i.invoice_id = il.invoice_id
           WHERE billing_country = 'USA'
         ),
     usa_sum_quantity AS --Computing the total number of tracks sold in the USA
         (
          SELECT SUM(il.quantity)
            FROM invoice_line AS il
           INNER JOIN invoice AS i
              ON i.invoice_id = il.invoice_id
           WHERE i.billing_country = 'USA'
         )

--Finding out which genres sell the best in the USA

SELECT genre,
       SUM(quantity) AS tracks_sold,
       ROUND(CAST(SUM(quantity) AS FLOAT) / (SELECT *
                                               FROM usa_sum_quantity
                                            ) * 100, 2) AS percentage_sold
  FROM usa_tracks
 GROUP BY genre
 ORDER BY tracks_sold DESC;

Done.


genre,tracks_sold,percentage_sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on the sales of tracks across different genres in the USA, we should select the following albums to be added to the Chinook store.

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

It's worth highlighting that these three genres combined represent only 17% of total sales. It looks like it would be a better deal to find artists from the *'Rock'* genre since it accounts for 53% of sales by itself. 


## Analysing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase.

We are going to analyse the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [4]:
%%sql

-- Finding the total sales assigned to each sales support agent

SELECT e.first_name || ' ' || e.last_name AS sales_agent,
       e.hire_date,
       SUM(i.total) AS total_sales
  FROM invoice AS i
 INNER JOIN customer AS c
    ON c.customer_id = i.customer_id
 INNER JOIN employee AS e
    ON e.employee_id = c.support_rep_id
 WHERE e.title = 'Sales Support Agent'
 GROUP BY sales_agent
 ORDER BY total_sales DESC;

Done.


sales_agent,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


Although the top sales support agent (Jane Peacock) has sold 20% more than the bottom sales support agent (Steve Johnson), the variation is probably due to the differences in their hiring dates.


## Analysing Sales by Country

Next, we will analyse the sales data for customers from each different country. In particular, we will calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, we will group these customers as *'Other'* in our analysis.

In [5]:
%%sql

-- Joining tables and creating 'Other' category for countries with only one customer

WITH country_or_other AS
     (
      SELECT CASE
                 WHEN(SELECT COUNT(*)
                        FROM customer
                       WHERE country = c.country
                     ) = 1 THEN 'Other'
                 ELSE c.country
             END country,
             c.customer_id,
             il.invoice_id,
             il.unit_price
        FROM customer AS c
  INNER JOIN invoice AS i
          ON i.customer_id = c.customer_id
  INNER JOIN invoice_line AS il
       ON il.invoice_id = i.invoice_id
     )
    
-- Calculating sales data for customers by each country

SELECT country,
       number_customers,
       total_sales,
       avg_sales_per_customer,
       avg_order_value
  FROM (
        SELECT country,
               COUNT(DISTINCT(customer_id)) AS number_customers,
               SUM(unit_price) AS total_sales,
               SUM(unit_price) / COUNT(DISTINCT(customer_id)) AS avg_sales_per_customer,
               SUM(unit_price) / COUNT(DISTINCT(invoice_id)) AS avg_order_value,
               CASE
                   WHEN country = 'Other' THEN 1
                   ELSE 0
               END AS sort
          FROM country_or_other
         GROUP BY country
         ORDER BY sort ASC, number_customers DESC
       );

Done.


country,number_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486


Based on our results, there may be an opportunity in the following countries:

- Czech Republic
- United Kingdom
- India

As the amount of data from each of these countries is relatively low, the Chinook store should be prudent to spend too much money on new marketing campaigns. A better approach would be to run small campaigns in these countries, collecting and analysing the data of the new customers to make sure that these trends apply to new customers.


## Albums vs Individual Tracks

The Chinook store is setup in a way that allows customers to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management is currently considering changing its purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We will find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [6]:
%%sql

-- Selecting the first track of each invoice

WITH invoice_first_track AS
     (
      SELECT il.invoice_id AS invoice_id,
             MIN(il.track_id) AS first_track_id
        FROM invoice_line AS il
       GROUP BY il.invoice_id
     )

/* Categorizing each invoice as either album purchase or not
   Calculating number of invoices and percetage of invoices */

SELECT album_purchase,
       COUNT(invoice_id) AS number_of_invoices,
       CAST(COUNT(invoice_id) AS FLOAT) / (
                                           SELECT COUNT(*)
                                             FROM invoice
                                          ) AS percentage
  FROM(
       SELECT ift.*,
              CASE
                  WHEN
                      (
                       SELECT t.track_id
                         FROM track AS t
                        WHERE t.album_id = (SELECT t2.album_id
                                              FROM track AS t2
                                             WHERE t2.track_id = ift.first_track_id
                                           )
               
       
                       EXCEPT

                       SELECT il2.track_id
                         FROM invoice_line AS il2
                        WHERE il2.invoice_id = ift.invoice_id
                      ) IS NULL
                  AND
                      (
                       SELECT il2.track_id
                         FROM invoice_line AS il2
                        WHERE il2.invoice_id = ift.invoice_id
       
                       EXCEPT

                       SELECT t.track_id
                         FROM track AS t
                        WHERE t.album_id = (SELECT t2.album_id
                                              FROM track AS t2
                                             WHERE t2.track_id = ift.first_track_id
                                           )
                      ) IS NULL
                  THEN 'yes'
                  ELSE 'no'
              END AS album_purchase
       FROM invoice_first_track AS ift
  )
 GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percentage
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases represent 18.6% of purchases. Based on this data, it is recommendable for the Chinook store not to purchase only selected tracks from albums from record companies, since there is potential to lose one-fifth of revenue.